<a href="https://colab.research.google.com/github/vikirams/ChatGPT/blob/main/Linkedin_Keyword_search_for_Post.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import json
import time
from openpyxl import Workbook, load_workbook
import re
from datetime import datetime, timedelta
import os

pagination_token = ""
total_post_count = 350
start = 0
count = 10

def getProfileData():
    file_name = "#hashtag.xlsx"

    # Check if the file exists, if not create it with headers
    if not os.path.exists(file_name):
        wb = Workbook()
        ws = wb.active
        ws.title = "#saastr"
        headers = ["Author", "Type", "Description", "Profile URL", "Post Time", "Content", "Likes", "Comments", "Shares"]
        ws.append(headers)
    else:
        wb = load_workbook(file_name)
        ws = wb["#saastr"]
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",
        "csrf-token": "ajax:"  # Placeholder for the CSRF token
    }

    with requests.Session() as s:
        s.cookies['li_at'] = '----'
        s.cookies['JSESSIONID'] = "ajax:"
        s.headers.update(headers)

        global pagination_token, start, total_post_count

        while start < total_post_count:
            if pagination_token:
                url = f'https://www.linkedin.com/voyager/api/graphql?variables=(keywords:GITEX,paginationToken:{pagination_token},sortOrder:RELEVANCE,start:{start},count:{count})&queryId=voyagerFeedDashInterestUpdates.3f133c09101fcd6b10faaa5c1540b9f9'
            else:
                url = f'https://www.linkedin.com/voyager/api/graphql?includeWebMetadata=true&variables=(keywords:GITEX,sortOrder:RELEVANCE,start:{start},count:{count})&queryId=voyagerFeedDashInterestUpdates.3f133c09101fcd6b10faaa5c1540b9f9'
            # if start >= 301:
            #     break
            time.sleep(10)
            response = s.get(url)

            if response.status_code == 200:
                response_dict = response.json()
                extract_linkedin_posts(response_dict, ws)
                start += count
                #break
            else:
                print(f"Error: {response.status_code} - {response.text}")
                break
        # Save the workbook after all data has been processed
        wb.save(file_name)
        print(f"All data has been saved to {file_name}")

def parse_linkedin_time(time_str):
    now = datetime.now()

    # Parse the short form (e.g., "47m", "2h", "3d")
    match = re.match(r'(\d+)([mhdw])', time_str)
    if match:
        number, unit = match.groups()
        number = int(number)
        if unit == 'm':
            return now - timedelta(minutes=number)
        elif unit == 'h':
            return now - timedelta(hours=number)
        elif unit == 'd':
            return now - timedelta(days=number)
        elif unit == 'w':
            return now - timedelta(weeks=number)

    # Parse "Edited" or other non-time strings
    if 'Edited' in time_str:
        # If "Edited" is present, try to find a time before it
        match = re.match(r'(\d+)([mhdw])', time_str)
        if match:
            number, unit = match.groups()
            number = int(number)
            if unit == 'm':
                return now - timedelta(minutes=number)
            elif unit == 'h':
                return now - timedelta(hours=number)
            elif unit == 'd':
                return now - timedelta(days=number)
            elif unit == 'w':
                return now - timedelta(weeks=number)

    # If no match is found, return the current date
    print("Current time"+ time_str)
    return None

def extract_linkedin_posts(json_data, worksheet):
    global pagination_token, total_post_count

    data = json_data['data']['feedDashInterestUpdatesByInterestFeedByKeywords']
    metadata = data['metadata']
    pagination_token = metadata.get('paginationToken')
    pagination_expiry = metadata.get('paginationTokenExpiryTime')
    total_post_count = data['paging']['total']

    print(f"Total Post Count: {total_post_count}")
    print(f"Pagination Token: {pagination_token}")
    print(f"Pagination Token Expiry: {pagination_expiry}")
    print(f"Current Start: {start}")
    print("\n--- Posts ---\n")

    posts = data['elements']

    for post in posts:
        try:
            post_data = extract_post_info(post)
            if not post_data:
                continue
            worksheet.append(post_data)
        except Exception as e:
            print(f"Error processing post: {e}")
            break

def extract_post_info(post):
    actor = post.get('actor', {})

    time_str = actor.get('subDescription', {}).get('text', '')
    post_time = parse_linkedin_time(time_str)
    if post_time == None:
        return []
    formatted_post_time = post_time.strftime("%m/%d/%Y")
    print("\n")
    print(time_str + " - " + formatted_post_time)


    author_name = actor.get('name', {}).get('text', 'Unknown')
    author_description = actor.get('description', {}).get('text', 'No description')

    is_company = 'company' in actor.get('followAction', {}).get('unfollowTrackingActionType', '').lower() if actor.get('followAction') else False
    entity_type = "Company" if is_company else "Person"

    profile_url = actor.get('navigationContext', {}).get('actionTarget', 'No URL')

    # Safely extract content
    commentary = post.get('commentary', {})
    if isinstance(commentary, dict):
        content = commentary.get('text', {}).get('text', 'No content')
    else:
        content = 'No content'

    social_detail = post.get('socialDetail', {})
    reaction_counts = social_detail.get('totalSocialActivityCounts', {})
    num_likes = reaction_counts.get('numLikes', 0)
    num_comments = reaction_counts.get('numComments', 0)
    num_shares = reaction_counts.get('numShares', 0)



    return [author_name, entity_type, author_description, profile_url, formatted_post_time, content[:1000], num_likes, num_comments, num_shares]

# The parse_linkedin_time function remains the same as in the previous example

getProfileData()

Total Post Count: 475
Pagination Token: 33189793-1728626112024-1cc3e1850c50f48235ef2de09ce9ce42
Pagination Token Expiry: 1728629712024
Current Start: 0

--- Posts ---



1m •    - 10/11/2024
Current timePromoted


2m •    - 10/11/2024


7m •    - 10/11/2024


10m •    - 10/11/2024
Current timePromoted
Total Post Count: 475
Pagination Token: 33189793-1728626112024-1cc3e1850c50f48235ef2de09ce9ce42
Pagination Token Expiry: 1728629712024
Current Start: 10

--- Posts ---



20m •    - 10/11/2024


29m •    - 10/11/2024


40m •    - 10/11/2024


44m •    - 10/11/2024


54m •    - 10/11/2024


55m •    - 10/11/2024
Current timePromoted


55m •    - 10/11/2024


55m •    - 10/11/2024
Total Post Count: 475
Pagination Token: 33189793-1728626112024-1cc3e1850c50f48235ef2de09ce9ce42
Pagination Token Expiry: 1728629712024
Current Start: 20

--- Posts ---



55m •    - 10/11/2024


55m •    - 10/11/2024


56m •    - 10/11/2024


1h •    - 10/11/2024


1h • Edited •    - 10/11/2024
Current timePromote